Import all necessary libraries and install everything you need for training:

In [1]:
# install the libraries necessary for data wrangling, prediction and result analysis
import json
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score,precision_score, recall_score
import torch
from numba import cuda
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [2]:
# Install transformers
# (this needs to be done on Kaggle each time you start the session)
#!pip install -q transformers

In [2]:
# Install the simpletransformers
#!pip install -q simpletransformers
from simpletransformers.classification import ClassificationModel

In [4]:
# Install wandb
#!pip install -q wandb

In [3]:
import wandb

In [4]:
# Login to wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tajak (use `wandb login --relogin` to force relogin)


True

In [21]:
# Clean the GPU cache

cuda.select_device(0)
cuda.close()
cuda.select_device(0)
torch.cuda.empty_cache()


RuntimeError: CUDA error: invalid argument
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

### Import the data

In [12]:
# GINCO full set (all labels except the labels with less than 10 instances)
train_df = pd.read_csv("data/GINCO-full-set-train.csv", index_col = 0)
dev_df = pd.read_csv("data/GINCO-full-set-dev.csv", index_col = 0)
test_df = pd.read_csv("data/GINCO-full-set-test.csv", index_col = 0)

print("GINCO train shape: {}, Dev shape: {}, Test shape: {}.".format(train_df.shape, dev_df.shape, test_df.shape))

GINCO train shape: (579, 2), Dev shape: (193, 2), Test shape: (193, 2).


In [13]:
train_df.head()

,text,labels
680,CD PREDVAJALNIK <p/> CD predvajalnik <p/> CD p...,Promotion of a Product
769,Objavljeno: 20 Mar 2010 13:08 Naslov sporočila...,Forum
613,Nazadnje urejal/a kaela 19 Nov 2011 19:16; sku...,Forum
931,Katero hrano je dobro jesti skupaj in katero n...,Instruction
485,"Narišem navpično črto, vzporedno z levim robom...",Instruction


## Training and saving

We will use the multilingual XLM-RoBERTa model
https://huggingface.co/xlm-roberta-base

In [14]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [15]:
# Create a list of labels
LABELS = train_df.labels.unique().tolist()
LABELS

['Promotion of a Product',
 'Forum',
 'Instruction',
 'News/Reporting',
 'Opinionated News',
 'Other',
 'List of Summaries/Excerpts',
 'Invitation',
 'Opinion/Argumentation',
 'Legal/Regulation',
 'Information/Explanation',
 'Review',
 'Promotion of Services',
 'Promotion',
 'Announcement',
 'Call',
 'Correspondence']

In [16]:
# Initialize Wandb
wandb.init(project="GINCO-hyperparameter-search", name="hyperparameter-search")

In [17]:
# Calculate how many steps will each epoch have
# Num steps in epoch = training samples / batch size
steps_per_epoch = int(579/8)
steps_per_epoch

72

I evaluated per every 10th epoch - per 720 steps. I first trained the model while evaluating it to find the optimal number of epochs and then trained it again and saved it.

In [18]:
# Create a TransformerModel
roberta_base_model = ClassificationModel(
        "xlmroberta", "xlm-roberta-base",
        num_labels=len(LABELS),
        use_cuda=True,
        args= {
            "overwrite_output_dir": True,
            "num_train_epochs": 90,
            "train_batch_size":8,
            "learning_rate": 1e-5,
            # Use these parameters if you want to evaluate during training
            "evaluate_during_training": True,
            "evaluate_during_training_steps": steps_per_epoch*10,
            "evaluate_during_training_verbose": True,
            "use_cached_eval_features": True,
            'reprocess_input_data': True,
            "labels_list": LABELS,
            # The following parameters (no_cache, no_save) are commented out if I want to save the model
            "no_cache": True,
            # Disable no_save: True if you want to save the model
            "no_save": True,
            "max_seq_length": 512,
            "save_steps": -1,
            # Only the trained model will be saved - to prevent filling all of the space
            "save_model_every_epoch":False,
            "wandb_project": 'GINCO-hyperparameter-search',
            "silent": True,
            }
        )

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [20]:
# Train the model and evaluate during training
roberta_base_model.train_model(train_df, eval_df = dev_df)

/home/tajak/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 31.75 GiB total capacity; 7.18 GiB already allocated; 20.75 MiB free; 7.22 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [24]:
# Save the trained model to Wandb
run = wandb.init(project="FTD-learning-manual-hyperparameter-search", entity="tajak", name="saving-trained-model")
trained_model_artifact = wandb.Artifact("FTD-classifier", type="model", description="a model trained on the FTD dataset")
trained_model_artifact.add_dir("outputs")
run.log_artifact(trained_model_artifact)

wandb: Adding directory to artifact (./outputs)... Done. 2.7s


In [ ]:
# Clean the GPU cache
cuda.select_device(0)
cuda.close()
cuda.select_device(0)
torch.cuda.empty_cache()